In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
from scipy.spatial.distance import cdist
import nibabel as nib
from nibabel.affines import apply_affine


reference_cifti_path = Path(
    "/mnt/cache/pfm_python/"
    "/sub-ME01_task-rest_concatenated_and_demeaned_32k_fsLR.dtseries.nii"
)
surface_files = {
    'lh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.L.midthickness.32k_fs_LR.surf.gii"
    ),
    'rh': Path(
        "/mnt/brunodata/open_data/ds005118/derivatives/sub-ME01/fs_LR/fsaverage_LR32k"
        "/ME01.R.midthickness.32k_fs_LR.surf.gii"
    ),
}


In [ ]:
# Get gifti coordinates to calculate Euclidean distance between them.

# Load the reference Cifti2 image for its brain axis
ref_img = nib.cifti2.Cifti2Image.from_filename(reference_cifti_path)
brain_ax = ref_img.header.get_axis(1)
print(f"Length of cifti2 brain_axis: {len(brain_ax)}")

# Extract the 3D Cartesian coordinates of all surface vertices
lh_surface_img = nib.gifti.gifti.GiftiImage.from_filename(surface_files['lh'])
rh_surface_img = nib.gifti.gifti.GiftiImage.from_filename(surface_files['rh'])

# Extract the vertex indices into the mapped BOLD data
anat_map = {
    'CortexLeft': 'CIFTI_STRUCTURE_CORTEX_LEFT',
    'CortexRight': 'CIFTI_STRUCTURE_CORTEX_RIGHT',
}
lh_surf_anat = lh_surface_img.darrays[0].metadata.get('AnatomicalStructurePrimary', '')
lh_surf_idx = brain_ax[brain_ax.name == anat_map[lh_surf_anat]]
lh_surf_coords = lh_surface_img.darrays[0].data[lh_surf_idx.vertex, :]
print(f"Just vertices in {str(type(lh_surf_idx))} {lh_surf_anat}: {len(lh_surf_idx)}")
rh_surf_anat = rh_surface_img.darrays[0].metadata.get('AnatomicalStructurePrimary', '')
rh_surf_idx = brain_ax[brain_ax.name == anat_map[rh_surf_anat]]
rh_surf_coords = rh_surface_img.darrays[0].data[rh_surf_idx.vertex, :]
print(f"Just vertices in {str(type(rh_surf_idx))} {rh_surf_anat}: {len(rh_surf_idx)}")

# Get the subcortical voxels, too, from a volumetric grid rather than vertices.
# Note that python's voxel locations are consistently shifted relative to
# matlab's. Python's x values are ml+2mm, y=ml+2mm, z=ml-2mm.
# Maybe 0-based vs 1-based indexing, then multiplied by the affine?
# Maybe it's start of voxel vs end of voxel, not center?
# It's all relative, so the subcortex-to-subcortex distances are identical,
# and distance differences are only between subcortical and cortical.
# If I add one to all the voxel coordinates before applying the affine,
# my coordinates match those in Lynch's matlab code perfectly. This sounds
# suspect, but I'm not sure how to validate the "TRUE" location of
# a voxel. One way would be to ask whether either the python-style
# or matlab-style of applying an affine results in a leftward or rightward
# bias. We test that here.
ctx_labels = list(anat_map.values())
sc_coords_ml_style = apply_affine(
    brain_ax.affine, brain_ax.voxel[~np.isin(brain_ax.name, ctx_labels)] + 1,
)
sc_coords_py_style = apply_affine(
    brain_ax.affine, brain_ax.voxel[~np.isin(brain_ax.name, ctx_labels)],
)

print("Nifti subcortical coordinates (python): "
      f"{sc_coords_py_style.shape}")
print("Nifti subcortical coordinates (matlab): "
      f"{sc_coords_ml_style.shape}")
print("Cifti cortical coordinates: "
      f" = {lh_surf_coords.shape} & {rh_surf_coords.shape}")

whole_brain_coordinates = np.vstack([
    lh_surf_coords, rh_surf_coords, sc_coords_py_style
])
print("Whole brain coordinates: "
      f" = {whole_brain_coordinates.shape}")


The affine first multiplies x by -2, then shifts x by +90,
multiplies y by +2, then shifts y by -126,
multiplies z by +2, then shifts z by -72.
So a voxel at (10, 20, 30) would be scaled to (-20, 40, 60),
then shifted to (70, -86, -12). Adding one to the original
coordinates would result in (68, -84, -10).

Increasing z (A-P) would have little effect on average sc-ctx distance.
Increasing y (I-S) would reduce teh sc-ctx distance for both hemispheres.
Decreasing x (L-R) would cause a 2mm leftward bias in sc-ctx distances.
Do we see that adding 1 to python's coordinates corrected a rightward
bias or caused a leftward bias?   

In [ ]:
print(apply_affine(
    brain_ax.affine, np.array([[10, 20, 30, ], [11, 21, 31, ], [0.0, 0.0, 0.0, ]]),
))

In [ ]:
# Calculate Euclidean distances from each subcortical voxel to each
# cortical vertex
for (style, sc_coords) in [
    ("python", sc_coords_py_style), ("matlab", sc_coords_ml_style),
]:
    sc_to_lh_dist = np.uint8(
        np.clip(cdist(sc_coords, lh_surf_coords) + 0.5, 0, 255)
    )
    sc_to_rh_dist = np.uint8(
        np.clip(cdist(sc_coords, rh_surf_coords) + 0.5, 0, 255)
    )
    
    print(f"For {style}, "
          "the mean distance from sc to left cortex is "
          f"{np.mean(sc_to_lh_dist):0.2f}; "
          "and from sc to right cortex is "
          f"{np.mean(sc_to_rh_dist):0.2f}.")
    

Applying the affine to raw coordinates resulted in a 1.1mm rightward bias.
Adding 1 to the coordinates shifted subcortex leftward, slightly over-correcting the bias to -0.3mm.
It appears that adding 1 balances the left/right cortical distances and is probably correct.